Student Name: Joel Amarou Heuer

Student ID: 202102201

# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import math     

# for tokenization use nltk
import time
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/korosu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# (Hyper)Parameters

In [2]:
# TODO Hyperparameters
# input_size = 
output_size = 10 # -> number of neurons in output-layer (= for each possible rating?)
# [hidden_layers_sizes] = 
# learning_rate = 
number_of_epochs = 1 # how is every sample used for pass/forward/backward

# Parameters
train_data_path = "./data/drugLibTrain_raw.tsv" # please use relative path like this
test_data_path = "./data/drugLibTest_raw.tsv" # please use relative path like this

In [3]:
'''
EXAMPLE

inputs = [1,2,3,4]    das sind 4 Eingaben, also entweder 
						-> 4 Features für Input-Layer oder 
						-> 4 Neuronen der davorigen Layer

weights = [
			[w11,w12,w13,w14]       -> Neuoron1 hat 4 Weights, also von jedem Neuron der davorigen Schicht eins
			[w21,w22,w23,w24]		-> Neuoron2 hat 4 Weights, also von jedem Neuron der davorigen Schicht eins
		]
	  2x4
	#NeuronenProHiddenLayer x #Inputs
'''



def forward_pass(data):
	'''
	@param data is comment/review in string format
	-> for a string s, one-hot-encoding makes matrix with dimensions:     {#words-in-string}         x     {#all-words-in-dataset = input-size}

	multiply with weights-matrix, which has dimensions:				:     {#neuron-in-hidden-layeri} x     {#input-size}


	Formula1: 	weights     *      inputs       +     biases
	       	    {#n/l}x{#ip}       {#ip}{??}          {#ip}{??}


	

	
	'''

	print(data)
	# LayerInput --> LayerHidden1


	# LayerHidden2 --> LayerHidden2

	# LayerHidden2 --> LayerOutput
	pass


def train(train_data, train_labels, valid_data, valid_labels):

	for epoch in range(number_of_epochs):
		index = 0

#		# Same thing about [hidden_layers] mentioned above is valid here also
#		for data, labels in zip(train_data, train_labels):
#			predictions, [hidden_layers] = forward_pass(data)
#			loss_signals = derivation_of_loss_function(labels, predictions)
#			backward_pass(data, [hidden_layers], predictions, loss_signals)
#			loss = loss_function(labels, predictions)
#
#			if index % 400 == 0: # at each 400th sample, we run validation set to see our model's improvements
#				accuracy, loss = test(valid_data, valid_labels)
#				print("Epoch= "+str(epoch)+", Coverage= %"+ str(100*(index/len(train_data))) + ", Accuracy= "+ str(accuracy) + ", Loss= " + str(loss))
#
#			index += 1
#
#	return losses


# Main

In [4]:
input_features =  'commentsReview'

###########################################
################ Load Data ################
###########################################
# load data
train_data = pd.read_csv(train_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')
# drop nans
train_data.dropna(subset = [input_features], inplace=True)
test_data.dropna(subset = [input_features], inplace=True)
# lowercase all comments
train_data[input_features] = train_data[input_features].str.lower()
test_data[input_features] = test_data[input_features].str.lower()

In [5]:
############################################################
################ Create directory for words ################
############################################################
# merge data
df_data =  pd.concat([train_data,test_data], axis=0)
# create dictonary
dictonary_words = df_data["commentsReview"].apply(nltk.word_tokenize)
dictonary_words = dictonary_words.values.tolist()
# flat lists
dictonary_words = [item for sublist in dictonary_words for item in sublist]
# drop duplicates
dictonary_words = list(set(dictonary_words))
# number of all words
dict_size = len(dictonary_words)

print(f"Number of words {dict_size}")

Number of words 11584


In [6]:
##############################################
################ Tokenization ################
##############################################
train_data_tokenized:pd.DataFrame = train_data["commentsReview"].apply(nltk.word_tokenize)
test_data_tokenized:pd.DataFrame = test_data["commentsReview"].apply(nltk.word_tokenize)

In [7]:
##################################################
################ One-Hot-Encoding ################
##################################################

# tokenized sentence to onehot
def tokenized_to_onehot(tokenized_sentence):   
    one_hot_matrix = []
    for word in tokenized_sentence:
        # create one-hot array for word
        one_hot_word = np.zeros(shape=(dict_size))
        i_hot = dictonary_words.index(word)
        # set the 1 at index of dictonary
        one_hot_word[i_hot] = 1
        one_hot_matrix.append(one_hot_word)
    return one_hot_matrix


def split_computation(tokenized):
    '''
    just a support function. 
    somehow my computer crashes when i try to compute everthing in one operation
    '''
    size = len(tokenized)
    chunks = range(0,size,math.floor(size/15))
    splits = len(chunks)

    hot_list = [] 
    for i in range(0, splits):
        print(f'{i}/{splits}')
        time.sleep(10)
        if i==0: # first
            continue
        elif i == (splits -1): # last
            hot_list.append(tokenized.iloc[chunks[i-1]:size].apply(tokenized_to_onehot))
        else: # normal
            hot_list.append(tokenized.iloc[chunks[i-1]:chunks[i]].apply(tokenized_to_onehot))

    return pd.concat(hot_list, axis=0)



In [9]:
train_data_onehot = split_computation(train_data_tokenized)

# merge into dataframe
train_data[f'{input_features}_onehot'] = train_data_onehot
print("Done")

0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16


: 

: 

In [8]:
test_data_onehot = split_computation(test_data_tokenized)

# merge into dataframe
test_data[f'{input_features}_onehot'] = test_data_onehot

0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16


In [15]:
#################################################
################ Create datasets ################
#################################################
# split into train & test    and    feature & y columns
train_x = train_data[input_features]
train_y = train_data['rating']
test_x = test_data[input_features]
test_y = test_data['rating']



#####################################################################
################ # ONE-HOT for output (classes 1-10) ################
#####################################################################
# creating one-hot vector notation of labels. 
# (Labels are given numeric in the dataset)
new_train_y = np.zeros(shape=(len(train_y), output_size))
new_test_y = np.zeros(shape=(len(test_y), output_size))

# add 1's into one-hot-matrix
for i in range(len(train_y)) :    
    new_train_y[i][train_y[i]-1] = 1

for i in range(len(test_y)):
    new_test_y[i][test_y[i]-1] = 1

train_y = new_train_y
test_y = new_test_y


##################################################################
################ split into validation & training ################
##################################################################
# Split training data into [a] Training (75%) and [b] validation (25%)
valid_x = np.asarray(train_x[int(0.75*len(train_x)):-1])
valid_y = np.asarray(train_y[int(0.75*len(train_y)):-1])
train_x = np.asarray(train_x[0:int(0.75*len(train_x))])
train_y = np.asarray(train_y[0:int(0.75*len(train_y))])



#######################################
################ train ################
#######################################
# TODO
train(train_x, train_y, valid_x, valid_y)

######################################
################ test ################
######################################
# TODO 
#print("Test Scores:")
#print(test(test_x, test_y))

KeyError: 147